In [106]:
import pandas as pd
import numpy as np
import pandas_profiling
import sys
import os

from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 2

from src.data_processing import *
from src.utils import *

from sklearn.feature_selection import SelectKBest, chi2, f_classif

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Read the data
path = '../data/CustomerData_LeadGenerator.csv'
data_raw = load_raw(path)

# Check dtypes
data_raw.dtypes

fakeID                                             object
b_specialisation_i                                  int64
b_specialisation_h                                  int64
b_specialisation_g                                  int64
b_specialisation_f                                  int64
b_specialisation_e                                  int64
b_specialisation_d                                  int64
b_specialisation_c                                  int64
b_specialisation_b                                  int64
b_specialisation_a                                  int64
b_specialisation_j                                  int64
q_OpeningDays                                       int64
q_OpeningHours                                     object
q_2017 Average Household Size                     float64
q_2017 Total Households                             int64
q_2017 HHs: 5th Quintile (68.759 and above)         int64
q_5th Quint by Total HH                           float64
q_2017 Purchas

In [3]:
# Check why Opening hours is type object
mask = pd.to_numeric(data_raw['q_OpeningHours'], errors='coerce').isna()
print(f"number of non numeric values: {sum(mask)}")

number of non numeric values: 1


In [4]:
# Account for non numeric value in data_raw['q_OpeningHours']
data_clean = clean(data_raw)

In [5]:
# Use pandas profiling to interactively explore raw data in html report
profiling_reports(data_raw, "zeiss_raw_data")

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Notes of first report
- Data fake id column is unique so no duplicated potential customer
- Data set might be a list of offices of doctors, due to specialization columns
- Only a small fraction of 95 have been contacted, and 57 bought something if we assume that value 1 stand for a true value in b_in_kontakt_gewesen and b_gekauft_gesamt
- A lot of zeros in opening days (26,6%) --> If the assumption that each row retapresents a doctors office is true this might either be wrong data or it might mean that the office is not existing anymore
- Opening Hours had a issue with a non numeric value
- High correlation of numeric data

In [6]:
# Split data in train and secret test data
train, secret =  split_train_secret(data_clean)

In [7]:
# Create pandas profiling reports for raw data and secret data
reports = {'zeiss_train': train, 'zeiss_secret': secret}
for key, value in reports.items():
    profiling_reports(value, key)
    

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Summarize dataset:   0%|          | 0/39 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Notes of train and secret report
- Distribution of specialisation g,f and j heavily differ from train and test
- In train still 27.4% of data with 0 Opening Days & 27.4% in opening hours --> Assumption --> Missing data --> Therefore impute 
- Clean opening hours in secret test set
- Maybe Check for outlier and irrealistic values --> No completely unrealistic values for all other numerical variables 
- Other distributions of features look quite similar (eyeball estimate)

# Deep Dive into correlation of numeric features

In [8]:
# List with numeric features ordered by assumed similarity
num_cols = [
    'q_OpeningDays',
    'q_OpeningHours',
    'q_2017 Average Household Size',
    'q_2017 Total Households',
    'q_2017 HHs: 5th Quintile (68.759 and above)',
    'q_2017 Total Population',
    'q_2017 Pop 15+/Edu: University, Fachhochschule',
    'q_Uni by Total Pop',
    'q_5th Quint by Total HH',
    'q_2017 Purchasing Power: Per Capita',
    'q_2017 Personal Care: Per Capita',
    'q_2017 Personal Effects: Per Capita',
    'q_2017 Medical Products: Per Capita',
]

# Get correlation matrix
corr = data_clean[num_cols].corr()

# Plot correlation matrix
corr.style.background_gradient(cmap='coolwarm').format(precision=2)

,q_OpeningDays,q_OpeningHours,q_2017 Average Household Size,q_2017 Total Households,q_2017 HHs: 5th Quintile (68.759 and above),q_2017 Total Population,"q_2017 Pop 15+/Edu: University, Fachhochschule",q_Uni by Total Pop,q_5th Quint by Total HH,q_2017 Purchasing Power: Per Capita,q_2017 Personal Care: Per Capita,q_2017 Personal Effects: Per Capita,q_2017 Medical Products: Per Capita
q_OpeningDays,1.00,0.90,0.01,-0.04,-0.04,-0.04,-0.03,-0.01,-0.06,-0.09,-0.06,-0.11,-0.07
q_OpeningHours,0.90,1.00,-0.06,0.07,0.07,0.07,0.08,0.07,-0.10,-0.05,-0.00,-0.09,-0.04
q_2017 Average Household Size,0.01,-0.06,1.00,-0.67,-0.67,-0.67,-0.66,-0.82,0.51,-0.54,-0.82,-0.28,-0.57
q_2017 Total Households,-0.04,0.07,-0.67,1.00,1.00,1.00,1.00,0.80,-0.28,0.38,0.61,0.10,0.01
q_2017 HHs: 5th Quintile (68.759 and above),-0.04,0.07,-0.67,1.00,1.00,1.00,1.00,0.80,-0.25,0.41,0.63,0.13,0.02
q_2017 Total Population,-0.04,0.07,-0.67,1.00,1.00,1.00,1.00,0.80,-0.27,0.39,0.61,0.10,0.01
"q_2017 Pop 15+/Edu: University, Fachhochschule",-0.03,0.08,-0.66,1.00,1.00,1.00,1.00,0.81,-0.29,0.36,0.59,0.07,-0.00
q_Uni by Total Pop,-0.01,0.07,-0.82,0.80,0.80,0.80,0.81,1.00,-0.19,0.64,0.82,0.33,0.22
q_5th Quint by Total HH,-0.06,-0.10,0.51,-0.28,-0.25,-0.27,-0.29,-0.19,1.00,0.43,0.04,0.61,-0.18
q_2017 Purchasing Power: Per Capita,-0.09,-0.05,-0.54,0.38,0.41,0.39,0.36,0.64,0.43,1.00,0.91,0.92,0.49


# Check unrealistic values for opening days and opening hours

In [9]:
# Ceck if all values with opening hours = 0 have opening days = 0
cond_1 = data_clean['q_OpeningDays'] == 0
cond_2 = data_clean['q_OpeningHours'] == 0
print(f"# of records in whole data set with unlogic combination of q_OpeningDays and q_OpeningHours: "
      f"{data_clean[(cond_1 & ~cond_2) | (~cond_1 & cond_2)].shape[0]}")

# Check if opening hours per day has unrealistic values 
print(f"Max value of OpeningHours/OpeningDay: "
      f"{max(data_clean['q_OpeningHours']/data_clean['q_OpeningDays'])}")

# Check if there are any records in the training data having 0 opening hours but a true value in b_gekauft_gesamt
print(f"# of records with o q_OpeningHours but flagges as customer: "
      f"{train.loc[(cond_1 & cond_2), 'b_gekauft_gesamt'].sum()}")

# of records in whole data set with unlogic combination of q_OpeningDays and q_OpeningHours: 0
Max value of OpeningHours/OpeningDay: 12.0
# of records with o q_OpeningHours but flagges as customer:  17


# Impute 0 values of opening hour with mean()
- In real sceanrio a feedback loop with the owner of the data would be preferred 


In [71]:
# Impute data
data_imputed = impute(data_clean)

# Overwrite train and test split
train, secret =  split_train_secret(data_imputed)

# Manual feature selection of numeric features

In [72]:
# Manually select numerical features to dela with high multicollinearity
keep_num_features =  [
    'q_OpeningHours',
    'q_2017 Total Households',
    'q_2017 Purchasing Power: Per Capita',
    'q_2017 Medical Products: Per Capita',
    'q_5th Quint by Total HH'
]

# Get correlation matrix
corr_keep = data_imputed[keep_num_features].corr()

# Plot correlation matrix
corr_keep.style.background_gradient(cmap='coolwarm').format(precision=2)



,q_OpeningHours,q_2017 Total Households,q_2017 Purchasing Power: Per Capita,q_2017 Medical Products: Per Capita,q_5th Quint by Total HH
q_OpeningHours,1.00,0.11,0.05,0.03,-0.03
q_2017 Total Households,0.11,1.00,0.38,0.01,-0.28
q_2017 Purchasing Power: Per Capita,0.05,0.38,1.00,0.49,0.43
q_2017 Medical Products: Per Capita,0.03,0.01,0.49,1.00,-0.18
q_5th Quint by Total HH,-0.03,-0.28,0.43,-0.18,1.00
